In [1]:
from openai import OpenAI
import os
import json
import time

In [2]:
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

In [3]:
f = open('system_prompt.txt','r')
system_prompt = f.read()
f.close()
f = open('world_view', 'r')
world_view = f.read()
f.close()

In [12]:
# # create new assistant object
# assistant = client.beta.assistants.create(
#     name="Story Generator",
#     instructions = system_prompt,
#     response_format={"type": "json_object"},
#     model="gpt-4o",
# )
# ASSISTANT_ID = assistant.id

# Using pre-made assistant
STORY_ID = 'asst_QJULNvgbgB8cU1YgY5JzLAyr'
SKILL_ID = 'asst_JSej8B49OeeI7IwOUNp3dpKY'

In [5]:
player_status = '''{
  'Player Name': 'Argon', 
  'status': {'HP':200, 'MP':100, 
             'Strength':30, 'Skill':15, 
             'Dexterity': 5, 'Intelligence': 5, 
             'Luck': 5, 'Defense': 30, 
             'Speed': 5, 'Concentration': 20, 
             'Reaction': 15, 'MP Regeneration': 2, 'HP Regeneration': 2},
  'Gold': 1000, 
  'Level': 10, 
  'Gender': 'Male', 
  'Social Status': 'Commoner', 
  'Strength Level': '2-star Knight', 
  'Equipment': { 'Top': {'Item Name': 'Ordinary Leather Armor', 'Use Restriction': None, 'Effect': {'Defense':10, 'Dexterity': -3}}, 
                'Weapon': {'Item Name': 'Iron Sword', 'Use Restriction': {'Strength':15}, 'Effect': {'Strength':10}} } 
}'''

In [6]:
start_message = [
  {
    "type": "text",
    "text": world_view
  },
  {
    "type": "text",
    "text": player_status
  }
]

In [9]:
def wrap_text(text, width):
    for i in range(0, len(text), width):
        print(text[i:i + width])
        
def show_message(json_object):
  if 'Story' in json_object:
    wrap_text(json_object['Story'], 40)
    for idx, choice in enumerate(json_object['Choices']):
      print(f'{idx+1}: {choice["text"]} ({choice["status"]}), Gold:({choice["gold"]}), ({choice["next_type"]})')
  else:
    display(json_object)
    
def run_thread(user_message, thread, assistant_id):
  run = submit_message(user_message, thread, assistant_id)
  run = wait_run(run, thread)
  messages = get_message(thread)
  text = messages.data[0].content[0].text.value
  json_object=json.loads(text)
  show_message(json_object)
  return json_object


def start_thread(thread, start_message, assistant_id):
  # create thread and add first message in thread
  run_thread(start_message, thread, assistant_id)

  for i in range(5):
    user_message = input()
    print(user_message)
    if user_message == "quit":
      break
    response = run_thread(user_message, thread, assistant_id)
    if 'Combat' in response:
      run_thread(str(response['Combat']), thread, SKILL_ID)


def submit_message(user_message, thread, assistant_id):
  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
  )
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )
  return run


def wait_run(run, thread):
  while run.status == "queued" or run.status == "in_progress":
    # connect thread and assistant
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    time.sleep(0.5)
  return run

def get_message(thread):
  return client.beta.threads.messages.list(thread_id=thread.id)

In [13]:
thread = client.beta.threads.create()

In [14]:
start_thread(thread,start_message,STORY_ID)

Argon stood on the threshold of the Fore
st of Shadows, a remnant of ancient time
s and known for harboring many dark secr
ets. Among the murky paths and whisperin
g trees, Argon's mission was to locate t
he Silverleaf Elixir, a potent potion sa
id to cure any ailment. Legends told of 
it being guarded by dark forces, a chall
enge for even the staunchest knights and
 the most skilled mages. As he moved dee
per into the forest, the air grew colder
 and the shadows thicker. Suddenly, he f
elt a pair of eyes watching him.
1: Proceed cautiously and investigate the source of the gaze. ({'Concentration': 20, 'Luck': 5}), Gold:(0), (Combat)
2: Shout to challenge whoever is there. ({'Strength': 30, 'Defense': 30}), Gold:(0), (Combat)
3: Ignore it and continue on the path. ({}), Gold:(0), (Story)
2


{'Combat': {'Monster Name': 'Shadow Stalker',
  'Monster Description': 'A wraith-like being with a body of darkness, the Shadow Stalker silently haunts its prey, waiting for the opportune moment to strike. Its glowing red eyes pierce the gloom, and it moves with unnerving silence. It feeds on fear, growing stronger with each terrorized heart it encounters.',
  'status': {'HP': 180,
   'MP': 50,
   'Strength': 25,
   'Skill': 20,
   'Dexterity': 15,
   'Intelligence': 10,
   'Luck': 5,
   'Defense': 20,
   'Speed': 25,
   'Concentration': 15,
   'Reaction': 20,
   'MP Regeneration': 1,
   'HP Regeneration': 1}},
 'Reward': {'gold': 200, 'exp': 150},
 'Item': {'Item types': 'Potion',
  'Item Name': 'Shadow Essence',
  'Item Description': "A rare potion brewed from the essence of the Shadow Stalker. It temporarily increases the user's Speed and Reaction significantly.",
  'Use Restriction': {'Dexterity': 10},
  'Effect': {'Speed': 10, 'Reaction': 10}}}

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'content[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'content[0]', 'code': 'invalid_type'}}